## Importing all the modules

In [1]:
from qiskit_nature.settings import settings #General settings
from qiskit_nature.drivers import UnitsType #Units
from qiskit_nature.drivers.second_quantization import PySCFDriver #Driver = PySCF
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem #Problem = Electronic Structure

settings.dict_aux_operators = True

## Definition of the driver and the problem

In [13]:
driver = PySCFDriver(atom='O 0.00000 0.00000 0.11779; H 0.00000 0.75545 -0.47116; H 0.00000 -0.75545 -0.47116',
                     unit=UnitsType.ANGSTROM,
                     basis='sto3g')
problem = ElectronicStructureProblem(driver)


Once defined the driver and the problem we can extract the fermionic operators

In [17]:
## The driver computes the SCF in the molecule and give back all tis properties
properties = problem.driver.run()
print(properties)

ElectronicStructureDriverResult:
	DriverMetadata:
		Program: PYSCF
		Version: 2.0.1
		Config:
			atom=O 0.00000 0.00000 0.11779; H 0.00000 0.75545 -0.47116; H 0.00000 -0.75545 -0.47116
			unit=Angstrom
			charge=0
			spin=0
			basis=sto3g
			method=rhf
			conv_tol=1e-09
			max_cycle=50
			init_guess=minao
			max_memory=4000
			
	ElectronicBasisTransform:
		Initial basis: atomic
		Final basis: molecular
		Alpha coefficients:
		[0, 0] = 0.9941321417495028
		[0, 1] = -0.2327803662405383
		[0, 2] = 3.7275485610471973e-16
		[0, 3] = -0.10339754421426903
		[0, 4] = 1.576689827286286e-16
		[0, 5] = -0.13211337231776593
		[0, 6] = 1.027664324128167e-15
		[1, 0] = 0.026542153868198567
		[1, 1] = 0.8336572386130218
		[1, 2] = -1.988868805937764e-15
		[1, 3] = 0.5378518554054724
		[1, 4] = -8.384377909497122e-16
		[1, 5] = 0.882627014627006
		[1, 6] = -6.890714712947671e-15
		[2, 0] = 2.9852355105398514e-19
		[2, 1] = 1.298111858282172e-17
		[2, 2] = -2.4190875797675583e-16
		[2, 3] = 1.391739657

Now, if we want the second quantization operators wich can be transformed into Qbits, we can just call any property we want

In [23]:
second_q_ops = properties.second_q_ops()
print(second_q_ops)

{'ParticleNumber': FermionicOp([('+_0 -_0', (1+0j)), ('+_1 -_1', (1+0j)), ('+_2 -_2', (1+0j)), ('+_3 -_3', (1+0j)), ('+_4 -_4', (1+0j)), ('+_5 -_5', (1+0j)), ('+_6 -_6', (1+0j)), ('+_7 -_7', (1+0j)), ('+_8 -_8', (1+0j)), ('+_9...)], register_length=14, display_format='sparse'), 'ElectronicEnergy': FermionicOp([('+_0 +_1 -_2 -_6', (0.009205788187464755+0j)), ('+_0 +_1 -_3 -_5', (-0.0017030735691238984+0j)), ('+_0 -_1 +_2 -_6', (-0.03058438322440522+0j)), ('+_0 -_1 -_2 +_6', (0.02137859503694055+0j)), (...)], register_length=14, display_format='sparse'), 'DipoleMomentX': FermionicOp([('+_0 -_4', (0.0638775201534325+0j)), ('-_0 +_4', (-0.0638775201534325+0j)), ('+_1 -_4', (0.6193945919764122+0j)), ('-_1 +_4', (-0.6193945919764122+0j)), ('+_3 -_4', (0.1697746842915412+0j)), ('...)], register_length=14, display_format='sparse'), 'DipoleMomentY': FermionicOp([('+_0 -_2', (0.039954756925553+0j)), ('+_0 -_6', (0.05602601320586889+0j)), ('-_0 +_2', (-0.03995475692555301+0j)), ('-_0 +_6', (-0.05

All the information is stored into a dictionary so, if we want the Hamiltonian in the Fock space, we just need the electronic energies times the fock vectors

In [24]:
hamiltonian = second_q_ops['ElectronicEnergy']
print(hamiltonian)

Fermionic Operator
register length=14, number terms=1085
  (0.009205788187464755+0j) * ( +_0 +_1 -_2 -_6 )
+ (-0.0017030735691238984+0j) * ( +_0 +_1 -_3 -_5 )
+ (-0.03058438322440522+0j) * ( +_0 -_1 +_2 -_6 )
+ (0.02137859503694055+0j) * ( +_0 -_1 -_2 + ...


The Hamiltonian has 1085 terms and the fock vectors has 14 spin orbitals, meaning that if we map 1 to 1 we will need 14 Qbits to define the Hamiltonian.
The numbers in the vector denotes the spin orbital (numbered from 0 to 13) and the + means a creation operator and the - an anihilation operator

## Inspecting the Ground state

To do that we can decide if we want to use all the 14 spin orbitals (full CI-like) or we prefer to constrain the active space (CASSCF like)

### 1. All the 14 spin orbitals

Transformation from fermions to spins (Qbits).
We need to use a Mapper, let's try with Jordan-Wigner with has a 1 to 1 correspondence between occupation number and spin

In [25]:
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

jw_mapper = JordanWignerMapper()
jw_converter = QubitConverter(jw_mapper)

In [26]:
qbit_op_jw = jw_converter.convert(hamiltonian)
print(qbit_op_jw)

-55.61212698432872 * IIIIIIIIIIIIII
+ 0.8061244983030669 * ZIIIIIIIIIIIII
+ 0.7795506530460394 * IZIIIIIIIIIIII
+ 0.11283410252808601 * ZZIIIIIIIIIIII
+ 1.3708602657008608 * IIZIIIIIIIIIII
+ 0.15010579383796732 * ZIZIIIIIIIIIII
+ 0.13767273399576 * IZZIIIIIIIIIII
+ 1.3154192445135562 * IIIZIIIIIIIIII
+ 0.13470402202907666 * ZIIZIIIIIIIIII
+ 0.120079254222685 * IZIZIIIIIIIIII
+ 0.16816470425994617 * IIZZIIIIIIIIII
+ 1.2099676812195401 * IIIIZIIIIIIIII
+ 0.11453098330997258 * ZIIIZIIIIIIIII
+ 0.1252339820842753 * IZIIZIIIIIIIII
+ 0.1500450589483724 * IIZIZIIIIIIIII
+ 0.13778361328171207 * IIIZZIIIIIIIII
+ 1.6565171062284563 * IIIIIZIIIIIIII
+ 0.14056082257827357 * ZIIIIZIIIIIIII
+ 0.12815064008520738 * IZIIIZIIIIIIII
+ 0.15073726050937974 * IIZIIZIIIIIIII
+ 0.13783959943970553 * IIIZIZIIIIIIII
+ 0.12523411590960745 * IIIIZZIIIIIIII
+ 0.02956709889517203 * IXIXIIIIIIIIII
+ 0.22409029139030445 * IXZXIIIIIIIIII
+ 0.024868507353965945 * ZXZXIIIIIIIIII
+ 0.02956709889517203 * IYIYIIIIIIIIII
+

The nomenclature of the Qbits is: weight * Pauli term

We can reduce the size of the problem considering symmetries in the Hilbert space of our system. For that we can use a **Parity Mapper** that can remove 2 qbits by exploiting the particle-conserving properties of the electronic structure problems:

In [31]:
from qiskit_nature.mappers.second_quantization import ParityMapper

parity_mapper = ParityMapper()
parity_converter = QubitConverter(parity_mapper, two_qubit_reduction=True)
## This converter will need as a second argument the number of particles, we can get that from the problem or the properties
qbit_op_parity = parity_converter.convert(hamiltonian,properties.get_property('num_particles'))
## Or directly from problem
qbit_op_parity = parity_converter.convert(hamiltonian,problem.num_particles) ## This solution requires re-run the driver
## in any case:
print(qbit_op_parity)
## This new qbit hamiltonian has 12 qbits instead of 14

-55.612126984328704 * IIIIIIIIIIII
+ 0.8061244983030665 * ZIIIIIIIIIII
+ 0.11283410252808597 * IZIIIIIIIIII
+ 0.7795506530460391 * ZZIIIIIIIIII
+ 0.13767273399575994 * ZIZIIIIIIIII
+ 1.3708602657008604 * IZZIIIIIIIII
+ 0.15010579383796727 * ZZZIIIIIIIII
+ 0.16816470425994612 * IZIZIIIIIIII
+ 1.3154192445135557 * IIZZIIIIIIII
+ 0.1347040220290766 * ZIZZIIIIIIII
+ 0.12007925422268495 * ZZZZIIIIIIII
+ 0.02956709889517202 * IXXIIIIIIIII
- 0.2240902913903044 * IYYIIIIIIIII
- 0.024868507353965934 * ZYYIIIIIIIII
- 0.024868507353965934 * IXXZIIIIIIII
- 0.2240902913903044 * ZXXZIIIIIIII
+ 0.02956709889517202 * ZYYZIIIIIIII
+ 0.007760397196672284 * XIIXIIIIIIII
+ 0.0008173771340911965 * XZIXIIIIIIII
+ 0.008577774330763474 * YZZYIIIIIIII
+ 0.039388387350848694 * YXYXIIIIIIII
+ 0.3688966747400324 * YXXYIIIIIIII
+ 0.02555473959821835 * XYXYIIIIIIII
- 0.04151667665923619 * YYYYIIIIIIII
+ (-0.23659608467896126-1.3877787807814457e-17j) * IIIIZIIIIIII
+ 0.13778361328171201 * IIZIZIIIIIII
+ 1.2099676812

Now, to get the ground state energy we will use the variational principle in a quantum algorithm.
Which means the variational equation will be classically evaluated but the expectation values will be calculated in the quantum computer side as the expectation value of the Pauli operator

In [36]:
## Factory settings
from qiskit.algorithms.optimizers import SLSQP # Optimizer
from qiskit.providers.aer import StatevectorSimulator, QasmSimulator ## Quantum computer simulator
## We need now to build an ansatz for the solver. In this case we will solve the ground state finding the minimum eigenvalues 
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import VQEUCCFactory ## Uses the variational quantum eigensolver with a Unitary Coupled Cluster ansatz

vqe_factory = VQEUCCFactory(quantum_instance=StatevectorSimulator() ,optimizer=SLSQP()) ## if we use instead the Qasm simulator we can analyze the noise

In [37]:
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
solver_full = GroundStateEigensolver(jw_converter, vqe_factory)
solver_parity = GroundStateEigensolver(parity_converter,vqe_factory)
result_full = solver_full.solve(problem)
result_parity = solver_parity(problem)
## This calculation will take a while with the simulator

KeyboardInterrupt: 

In [ ]:
#Print the results
print(result_full)
print(result_parity)

### 2. A Smaller active space
Let's pick an active space with 6 spin orbitals (2 electrons and 3 orbitals)

Qiskit nature has a active space selector implemented

In [45]:
from qiskit_nature.transformers.second_quantization.electronic import ActiveSpaceTransformer
transformer = ActiveSpaceTransformer(num_electrons=2,num_molecular_orbitals=3)
problem_active_space = ElectronicStructureProblem(driver,[transformer])
second_q_ops_cas = problem_active_space.second_q_ops()
hamiltonian_cas = second_q_ops_cas['ElectronicEnergy']
print(hamiltonian_cas)

Fermionic Operator
register length=6, number terms=33
  (0.03857537755951361+0j) * ( +_0 -_1 +_3 -_4 )
+ (-0.03857537755951358+0j) * ( +_0 -_1 -_3 +_4 )
+ (0.024323983164891053+0j) * ( +_0 -_2 +_3 -_5 )
+ (-0.024323983164891053+0j) * ( +_0 -_2 -_3 +_ ...


Now, that we have a very reduced system we can still apply the parity to get rid of two Qbits

In [59]:
qbit_op_parity_cas = parity_converter.convert(hamiltonian_cas,problem_active_space.num_particles)
print(qbit_op_parity_cas)
## This new setting uses only 4 Qbits

-0.0749793175380048 * IIII
+ (-0.6108686075981941+1.3877787807814457e-17j) * ZIII
+ (0.2883250096745555-1.3877787807814457e-17j) * IZII
- 0.571782052054844 * ZZII
+ 0.6108686075981941 * IIZI
- 0.1548356507578057 * ZIZI
+ 0.15618678962918991 * IZZI
- 0.14161494051448403 * ZZZI
- 0.2883250096745553 * IIIZ
+ 0.15618678962918991 * ZIIZ
- 0.22003977334376143 * IZIZ
+ 0.14731657838563847 * ZZIZ
+ (-0.5717820520548441-1.3877787807814457e-17j) * IIZZ
+ 0.14161494051448403 * ZIZZ
+ (-0.14731657838563847+1.3877787807814457e-17j) * IZZZ
+ 0.14936809358574782 * ZZZZ
+ 0.028780837986398192 * XIXI
- 0.028780837986398196 * XZXI
+ (0.028780837986398196+1.734723475976807e-18j) * XIXZ
- 0.028780837986398196 * XZXZ
+ 0.009643844389878394 * IXIX
+ (0.009643844389878394+8.673617379884035e-19j) * ZXIX
- 0.009643844389878394 * IXZX
+ (-0.009643844389878394-8.673617379884035e-19j) * ZXZX
+ 0.006080995791222761 * XXXX
+ (-0.006080995791222761-4.336808689942018e-19j) * YYXX
- 0.006080995791222761 * XXYY
+ 0.006

Now we can get the ground state energy using the same factory as above

In [61]:
solver_cas = GroundStateEigensolver(parity_converter,vqe_factory)
result_cas = solver_cas.solve(problem_active_space)
print(result_cas)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -84.153940801523
  - computed part:      -1.664736243477
  - ActiveSpaceTransformer extracted energy part: -82.489204558047
~ Nuclear repulsion energy (Hartree): 9.189222125372
> Total ground state energy (Hartree): -74.964718676152
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  0.67748018]
    - computed part:      [0.0  0.0  0.44301637]
    - ActiveSpaceTransformer extracted energy part: [0.0  0.0  0.2344638]
  > Dipole moment (a.u.): [0.0  0.0  -0.67748018]  Total: 0.67748018
                 (debye): [0.0  0.0  -1.72198269]  Total: 1.72198269
 


Doing our own ansatz and sending the calculation to a real quantum computer (simulator)

In [72]:
from qiskit.circuit.library import EfficientSU2 
ansatz = EfficientSU2(qbit_op_parity_cas.num_qubits, reps=3, entanglement='linear', insert_barriers=True)
#ansatz.decompose().draw('latex', style='iqx')

In [77]:
from qiskit import Aer # Backend for the simulation

backend = Aer.get_backend('aer_simulator_statevector')


Solving the problem using the Variational Quantum Eigensolver

In [78]:
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms import VQE

optimizer = SLSQP()

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend)

result = algorithm.compute_minimum_eigenvalue(qbit_op_parity_cas)
print(result.eigenvalue.real)

electronic_structure_result = problem.interpret(result)
print(electronic_structure_result)

-1.6631698731769151
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.663169873177
  - computed part:      -1.663169873177
~ Nuclear repulsion energy (Hartree): 9.189222125372
> Total ground state energy (Hartree): 7.526052252195
 
=== MEASURED OBSERVABLES ===
 
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 


In [82]:
from qiskit_nature.circuit.library import UCCSD
from qiskit_nature.circuit.library import HartreeFock
init_state = HartreeFock(problem_active_space.num_spin_orbitals,problem_active_space.num_particles, parity_converter)
ansatz = UCCSD(parity_converter,problem_active_space.num_particles, problem_active_space.num_spin_orbitals,reps=3,initial_state=init_state)

In [83]:
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms import VQE

optimizer = SLSQP()

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend)

result = algorithm.compute_minimum_eigenvalue(qbit_op_parity_cas)
print(result.eigenvalue.real)

electronic_structure_result = problem.interpret(result)
print(electronic_structure_result)

-1.6647359827300867
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.66473598273
  - computed part:      -1.66473598273
~ Nuclear repulsion energy (Hartree): 9.189222125372
> Total ground state energy (Hartree): 7.524486142642
 
=== MEASURED OBSERVABLES ===
 
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
